In [ ]:
import numpy as np
import keras
import pandas as pd 
import matplotlib.pyplot as plt 
import tensorflow as tf 
from PIL import Image 
import os
from sklearn.model_selection import train_test_split

In [ ]:
#collecting and storing images and labeles in lists.
data = []
labels = []
classes = 43
cur_path = "i:\\class\\Term3\\BDM3035\\Project\\"

for i in range(classes):
    path = os.path.join(cur_path,'Train',str(i))
    images =  os.listdir(path)

    for a in images:
        try:
            image = Image.open(path+'\\'+a)
            image = image.resize((30,30))
            image = np.array(image)
            data.append(image)
            labels.append(i)
        except:
            print("Error loading image")

data = np.array(data)
labels = np.array(labels)


In [ ]:
data
print("-------------------------------")
labels

In [ ]:
#1...validation of resizing of images

# Check sizes of data and labels
print("Data shape:", data.shape)
print("Labels shape:", labels.shape)

# Visualize a few images along with their labels
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 5, figsize=(4,3))
for i, ax in enumerate(axes.flat):
    ax.imshow(data[i])
    ax.set_title(f"Label: {labels[i]}")
    ax.axis('off')

plt.show()